In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os
sys.path.append(os.path.abspath('..'))
from backtracking import resolver
from prog_lineal import prog_lineal
from lectura_archivo import leer_archivo
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
from pandas.plotting import table
import glob

In [2]:
MAX_GRUPOS = 5

In [3]:

def experimentar():
    ruta_archivos = [r for r in glob.glob("../datos/*.txt") 
                 if os.path.basename(r) != "Resultados Esperados.txt"]
    registros = []

    for ruta in ruta_archivos:
        k, elementos = leer_archivo(ruta)
   
        if k >= MAX_GRUPOS:
            continue
        
        _, habilidades = zip(*elementos)
        
        # --- PL ---
        start_pl = time.perf_counter()
        pl_val, _, _ = prog_lineal(k, habilidades)
        elapsed_pl = time.perf_counter() - start_pl
        print(f"Archivo {ruta} con {len(habilidades)} elementos PL tardó {elapsed_pl:.2f}s")

        # --- Backtracking óptimo ---
        
        start_bt = time.perf_counter()
        opt_val, _ = resolver(k, habilidades)
        elapsed_bt = time.perf_counter() - start_bt
        print(f"Archivo {ruta} con {len(habilidades)} elementos tardó {elapsed_bt:.2f}s")
   
        ratio = pl_val / opt_val

        registros.append({
            "Valor PL": pl_val,
            "Valor BT": opt_val,
            "Ratio PL/BT": ratio,
        })

    df = pd.DataFrame(registros)
    return df


In [5]:
def guardar_tabla_png(df, filename="tabla.png", figsize=(12,4)):
    """
    Guarda un DataFrame como imagen PNG.
    
    Parámetros:
    - df: DataFrame a guardar
    - filename: nombre del archivo PNG
    - figsize: tamaño de la figura
    """
    #Redondeo Ratio PL/BT
    df["Ratio PL/BT"] = df["Ratio PL/BT"].round(6)

    fig, ax = plt.subplots(figsize=figsize)
    ax.axis('off')  # Oculta los ejes

    # Crear la tabla
    tbl = table(ax, df, loc='center', cellLoc='center', colWidths=[0.15]*len(df.columns))
    tbl.auto_set_font_size(False)
    tbl.set_fontsize(12)
    tbl.scale(1.2, 1.2)  # Ajusta tamaño

    plt.savefig(filename, bbox_inches='tight', dpi=200)
    plt.close()

In [ ]:

df = experimentar()
print(df)
print("FINISHED")
display(df)

guardar_tabla_png(df, filename="../informe/img/pl_vs_bt.png")

Archivo ../datos\10_3.txt con 10 elementos PL tardó 0.38s
Archivo ../datos\10_3.txt con 10 elementos tardó 0.00s
Archivo ../datos\14_3.txt con 14 elementos PL tardó 1.56s
Archivo ../datos\14_3.txt con 14 elementos tardó 0.22s
Archivo ../datos\14_4.txt con 14 elementos PL tardó 15.28s
Archivo ../datos\14_4.txt con 14 elementos tardó 0.43s
Archivo ../datos\15_4.txt con 15 elementos PL tardó 8.67s
Archivo ../datos\15_4.txt con 15 elementos tardó 0.34s
Archivo ../datos\20_4.txt con 20 elementos PL tardó 6803.72s
Archivo ../datos\20_4.txt con 20 elementos tardó 969.51s
Archivo ../datos\5_2.txt con 5 elementos PL tardó 0.08s
Archivo ../datos\5_2.txt con 5 elementos tardó 0.00s
Archivo ../datos\6_3.txt con 6 elementos PL tardó 0.15s
Archivo ../datos\6_3.txt con 6 elementos tardó 0.00s
Archivo ../datos\6_4.txt con 6 elementos PL tardó 0.15s
Archivo ../datos\6_4.txt con 6 elementos tardó 0.00s
Archivo ../datos\8_3.txt con 8 elementos PL tardó 0.18s
Archivo ../datos\8_3.txt con 8 elementos tardó

,Valor PL,Valor BT,Ratio PL/BT
0,385249.0,385249.0,1.0
1,15659106.0,15659106.0,1.0
2,15292055.0,15292055.0,1.0
3,4311889.0,4311889.0,1.0
4,21081875.0,21081875.0,1.0
5,1894340.0,1894340.0,1.0
6,1640690.0,1640690.0,1.0
7,807418.0,807418.0,1.0
8,4298131.0,4298131.0,1.0
